# Estimativa de Máxima Verossimilhança

Digamos que eu tenha alguns dados e queira modificar uma função para que ela se ajuste o máximo possível aos dados. Esqueça isso. Quero *maximizar a probabilidade* de a função gerar os dados que estou observando. Isso é exatamente o que significa a **estimativa de máxima verossimilhança**, onde encontramos a probabilidade conjunta de uma determinada função gerar alguns dados observados. Essa técnica é usada para ajustar distribuições de probabilidade a um determinado conjunto de dados, bem como para treinar algoritmos de aprendizado de máquina, como regressão linear e regressão logística.

Aprenderemos sobre a estimativa de máxima verossimilhança aplicando-a primeiro a uma distribuição normal. Em seguida, estenderemos essa estrutura para ajustar uma regressão linear. É claro que uma regressão linear pode ser ajustada usando mínimos quadrados, mas, como aprenderemos, a estimativa de máxima verossimilhança chegará a uma solução quase idêntica. Dado que a regressão linear é a base e o bloco de construção do aprendizado de máquina, isso conectará ideias sobre como a probabilidade desempenha um papel no aprendizado de máquina.

## Ajustando uma distribuição normal## Fitting a Normal Distribution

Vamos observar algumas medições de um processo de usinagem em uma oficina e plotá-las em uma reta numérica.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

X = np.array([54.4,55.8,55.9,58.5,59.1,61.1,61.3,61.7,62.8,
              63.2,63.5,63.6,63.6,63.7,64.0,64.2,65.0,65.0,
              65.7,66.0,66.2,66.5,67.7,67.7,68.2,69.4,69.5,
              70.2,70.5,71.8,72.8,72.8,73.8,76.2,77.4])

plt.plot(X, [0 for _ in X], 'o')
plt.show()

Suspeitamos que esses dados seguem uma distribuição normal e queremos ajustá-la a uma. Embora pudéssemos simplesmente pegar a média e o desvio padrão dos dados e usá-los em nossos parâmetros para uma distribuição normal, vamos adotar uma abordagem mais probabilística com estimativa de máxima verossimilhança. Outras distribuições, como a distribuição exponencial, precisariam adotar essa abordagem, assim como o ajuste de uma regressão linear ou logística.

Temos muito o que analisar. Vamos primeiro discutir a ideia de verossimilhança conjunta para esse propósito. Observe os pontos de dados e uma distribuição normal fornecida abaixo.

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

X = np.array([54.4,55.8,55.9,58.5,59.1,61.1,61.3,61.7,62.8,
              63.2,63.5,63.6,63.6,63.7,64.0,64.2,65.0,65.0,
              65.7,66.0,66.2,66.5,67.7,67.7,68.2,69.4,69.5,
              70.2,70.5,71.8,72.8,72.8,73.8,76.2,77.4])

mu, sigma = 65.696, 5.469

# plota a distribuição e os pontos
x_range = np.arange(start=-3 * sigma + mu, stop=3 * sigma + mu, step=.01)
plt.plot(X, [0 for _ in X], 'o')
plt.plot(x_range, norm.pdf(x_range, mu, sigma))
plt.show()

Imagine cada um desses pontos projetando-se para cima na curva, e os valores y resultantes são as probabilidades.

In [ ]:
plt.plot(x_range, norm.pdf(x_range, mu, sigma), color='orange')

# plota linhas
for x in X:
    plt.plot(np.array([x,x]),
              np.array([0, norm.pdf(x, mu, sigma)]),
             'bo', linestyle="--")

plt.show()

Os valores de y resultantes que se assemelham às probabilidades são o que queremos multiplicar, chamado de **probabilidade conjunta**. Podemos calculá-la usando a função `prod()` em um array NumPy.

In [ ]:
import numpy as np
from scipy.stats import norm

X = np.array([54.4, 55.8, 55.9, 58.5, 59.1, 61.1, 61.3, 61.7, 62.8,
              63.2, 63.5, 63.6, 63.6, 63.7, 64.0, 64.2, 65.0, 65.0,
              65.7, 66.0, 66.2, 66.5, 67.7, 67.7, 68.2, 69.4, 69.5,
              70.2, 70.5, 71.8, 72.8, 72.8, 73.8, 76.2, 77.4])

media, desvio = 65.696, 5.469

verossimilhancas = norm.pdf(X, media, desvio)
verossimilhanca_conjunta = norm.pdf(X, media, desvio).prod()

print(f"Verossimilhanças: {verossimilhancas}")
print(f"\nVerossimilhança Conjunta: {verossimilhanca_conjunta}")

Essa probabilidade conjunta provavelmente ficará muito pequena, como mostrado acima, pois estamos multiplicando muitas probabilidades. Nos bastidores, usamos a soma de logaritmos de verossimilhança (em vez da multiplicação) para evitar subfluxos de ponto flutuante, mas deixaremos o NumPy fazer esse trabalho. A questão é que queremos ajustar mu e sigma para maximizar essa probabilidade conjunta total (portanto, a estimativa de máxima verossimilhança).

Para aplicar a estimativa de máxima verossimilhança, precisamos primeiro aprender um algoritmo de otimização. Normalmente, usaríamos uma técnica como gradiente descendente ou gradiente descendente estocástico. No entanto, para evitar a necessidade de um curso intensivo de Cálculo (mais um curso da Anaconda sobre isso depois!), usaremos um algoritmo mais simples chamado hill climbing. O hill climbing nos permite usar uma busca aleatória, porém metódica, que faz ajustes aleatórios nos valores, mas aceita apenas valores que melhoram em direção a um objetivo. Nesse caso, o objetivo é a máxima verossimilhança conjunta total.

Lembre-se de que uma distribuição normal aceita os parâmetros *mu* $ \mu $ e *sigma* $ \sigma $. Faremos ajustes aleatórios repetidamente nesses dois parâmetros e aceitaremos apenas ajustes que melhorem a máxima verossimilhança total. Mas quais serão os ajustes aleatórios? Não queremos necessariamente usar força bruta com valores uniformemente aleatórios, mas podemos amostrar a partir de uma distribuição normal padrão (com média de 0 e desvio padrão de 1) para obtermos principalmente pequenos ajustes próximos de 0, mas ocasionalmente grandes ajustes na cauda. Observe a distribuição normal padrão abaixo.

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm

x_range = np.arange(start=-3, stop=3, step=.01)
plt.plot(x_range, norm.pdf(x_range, loc=0, scale=1))
plt.show()

Pode parecer bastante meta (e conceitualmente tortuoso) usarmos uma distribuição normal para ajustar aleatoriamente outra distribuição normal. Mas funciona! Lembre-se de que uma é a distribuição normal que estamos ajustando aos nossos dados, e outra distribuição normal gera ajustes aleatórios para $ \mu $ e $ \sigma $.

Vamos juntar esses conceitos e usar a técnica de hill climbing para executar a estimativa de máxima verossimilhança. Começaremos $ \mu $ em um dos pontos de dados e o desvio padrão em 1. Eles precisam começar aproximadamente em algum lugar próximo aos pontos e, então, a técnica de hill climbing os ajustará de acordo.

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# declarar os pontos de dados X
X = np.array([54.4, 55.8, 55.9, 58.5, 59.1, 61.1, 61.3, 61.7, 62.8,
              63.2, 63.5, 63.6, 63.6, 63.7, 64.0, 64.2, 65.0, 65.0,
              65.7, 66.0, 66.2, 66.5, 67.7, 67.7, 68.2, 69.4, 69.5,
              70.2, 70.5, 71.8, 72.8, 72.8, 73.8, 76.2, 77.4])

# iniciar a média no primeiro ponto de dado e o desvio padrão em 1
# não importa onde eles comecem, apenas que estejam
# em algum lugar dentro ou próximo ao conjunto de dados
media, desvio = X[0], 1

# gera um valor aleatório de uma distribuição normal padrão
# onde a média é 0 e o desvio padrão é 1
def ajuste_aleatorio(): return np.random.normal(loc=0, scale=1, size=1)[0]

# iniciar a melhor verossimilhança conjunta em 0
melhor_verossimilhanca_conjunta = 0

# fazer 10.000 ajustes aleatórios em média e desvio
for i in range(10_000):

    # ajustar aleatoriamente média e desvio
    ajuste_media, ajuste_desvio = ajuste_aleatorio(), ajuste_aleatorio()

    media += ajuste_media
    desvio += ajuste_desvio

    # calcular a nova verossimilhança conjunta de todos os pontos de dados
    nova_verossimilhanca_conjunta = norm.pdf(X, media, desvio).prod()

    # se a verossimilhança conjunta melhorar, manter as mudanças
    if nova_verossimilhanca_conjunta > melhor_verossimilhanca_conjunta:
        melhor_verossimilhanca_conjunta = nova_verossimilhanca_conjunta
        print(f"média, desvio -> {media}, {desvio}")
    else:
        # caso contrário, desfazer as mudanças
        media -= ajuste_media
        desvio -= ajuste_desvio

# plotar o resultado
intervalo_x = np.arange(start=-3 * desvio + media, stop=3 * desvio + media, step=.01)

plt.plot(X, [0 for _ in X], 'o')
plt.plot(intervalo_x, norm.pdf(intervalo_x, media, desvio))

plt.show()

Com base nas saídas de impressão e no gráfico acima, você pode ver que a curva se moveu efetivamente para se ajustar aos pontos. Observe também que, se tomarmos a média e o desvio padrão dos dados, eles correspondem ao que a estimativa de máxima verossimilhança encontrou.

In [ ]:
X.mean(), X.std()

Mas este não foi apenas um exercício acadêmico desnecessário. Você pode ajustar qualquer distribuição de probabilidade com base em dados usando esta técnica.

Agora, vamos estender essa ideia ao ajuste de uma regressão linear.

## Usando MLE para ajustar uma regressão linear

A seguir, vamos usar a estimativa de máxima verossimilhança para ajustar uma regressão linear, que consiste em ajustar uma reta passando por alguns pontos. Observe os dados abaixo.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

X = np.array([2.9,9.5,5.1,0.9,2.0,2.3,5.2,7.7,7.9,4.1,9.7,6.3,4.9,6.2,4.2,
              3.1,3.9,8.7,1.2,9.6,0.8,3.0,5.6,7.3,3.7,3.5,2.6,5.0,1.7,9.1,1.9,2.4,0.3,5.7,9.0])

Y = np.array([7.58,18.83,8.71,0.6,6.06,0.64,12.09,13.65,15.34,8.6,16.32,11.78,9.78,8.44,9.18,
              3.04,7.65,10.96,1.47,17.52,2.21,4.76,13.03,12.29,10.2,7.88,3.01,8.92,2.23,15.08,
              5.42,5.53,-0.5,11.66,14.7])


plt.plot(X, Y, 'o') 

Queremos traçar uma reta através desses pontos com o objetivo de analisar/prever a relação entre duas variáveis ​​$ X $ e $ Y $. Lembre-se da fórmula para uma função linear.

$
y = mx + b
$

In [ ]:
x_range = np.arange(start=X.min(), stop=X.max(), step=.01)
plt.plot(X, Y, 'o') 
plt.plot(x_range, 1.71160239*x_range + 0.53778288) 
plt.show()

Imagine que uma distribuição normal segue essa linha, onde a média seguirá essa linha, mas o desvio padrão permanecerá constante. Podemos tratar os valores reais de y como a variável de entrada (o que normalmente é *x* na PDF) e os valores previstos de y como a média. Isso significa que precisamos calcular a inclinação *m*, o intercepto *b* e o desvio padrão $ \sigma $. Como ajustar aleatoriamente três coeficientes de uma só vez é uma grande mudança, selecionaremos aleatoriamente apenas um deles para ajustar em cada iteração.

Além dessas poucas mudanças, estamos realmente ajustando uma distribuição normal usando a estimativa de máxima verossimilhança, como antes!

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# declarar os pontos de dados X e Y
X = np.array([2.9, 9.5, 5.1, 0.9, 2.0, 2.3, 5.2, 7.7, 7.9, 4.1, 9.7, 6.3, 4.9, 6.2, 4.2,
              3.1, 3.9, 8.7, 1.2, 9.6, 0.8, 3.0, 5.6, 7.3, 3.7, 3.5, 2.6, 5.0, 1.7, 9.1,
              1.9, 2.4, 0.3, 5.7, 9.0])

Y = np.array([7.58, 18.83, 8.71, 0.6, 6.06, 0.64, 12.09, 13.65, 15.34, 8.6, 16.32, 11.78, 9.78, 8.44, 9.18,
              3.04, 7.65, 10.96, 1.47, 17.52, 2.21, 4.76, 13.03, 12.29, 10.2, 7.88, 3.01, 8.92, 2.23, 15.08,
              5.42, 5.53, -0.5, 11.66, 14.7])

m, b, desvio = 1, 1, 1

# gera um valor aleatório de uma distribuição normal padrão
# onde a média é 0 e o desvio padrão é 1
def ajuste_aleatorio(): return np.random.normal(loc=0, scale=1, size=1)[0]

# iniciar a melhor verossimilhança conjunta em 0
melhor_verossimilhanca_conjunta = 0

# fazer 30.000 ajustes aleatórios em m, b e desvio
for i in range(30_000):

    # ajustar aleatoriamente m, b ou desvio
    ajuste = ajuste_aleatorio()
    coef_aleatorio = np.random.randint(0, 3)
    if coef_aleatorio == 0:
        m += ajuste
    elif coef_aleatorio == 1:
        b += ajuste
    elif coef_aleatorio == 2:
        desvio += ajuste

    # calcular a nova verossimilhança conjunta de todos os pontos de dados
    nova_verossimilhanca_conjunta = np.array([norm.pdf(y, m * x + b, desvio) for x, y in zip(X, Y)]).prod()

    # se a verossimilhança conjunta melhorar, manter as mudanças
    if nova_verossimilhanca_conjunta > melhor_verossimilhanca_conjunta:
        melhor_verossimilhanca_conjunta = nova_verossimilhanca_conjunta
        print(f"m, b, desvio -> {m}, {b}, {desvio}")
    else:
        # caso contrário, desfazer as mudanças
        if coef_aleatorio == 0:
            m -= ajuste
        elif coef_aleatorio == 1:
            b -= ajuste
        elif coef_aleatorio == 2:
            desvio -= ajuste

# plotar o resultado
intervalo_x = np.arange(start=X.min(), stop=X.max(), step=.01)

plt.plot(X, Y, 'o')
plt.plot(intervalo_x, m * intervalo_x + b)

plt.show()

Se compararmos com um método convencional de mínimos quadrados, obtemos um resultado quase idêntico.

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X.reshape([-1, 1]), Y)
print(f"m: {lr.coef_[0]}")                            
print(f"b: {lr.intercept_}")

## Exercício

Um bar está com trânsito lento às quartas-feiras e o gerente está pensando em adicionar uma promoção de happy hour.

img

Ele quer entender quantas chegadas de clientes ocorrem, em média, a cada hora. Ele pede ao barman que registre 12 chegadas de clientes e quanto tempo decorreu entre cada uma (em horas). Complete o código abaixo (substituindo os pontos de interrogação "?") para realizar a estimativa de máxima verossimilhança e encontrar o parâmetro `lambda` (o número médio de clientes por hora) na distribuição exponencial.

In [ ]:
import numpy as np
from scipy.stats import expon

# tempos observados entre cada cliente
X = np.array([0.27922493, 0.44124056, 0.50967118, 0.44413533, 0.67243048, 0.01870771,
              0.08661839, 0.29967495, 1.68386979, 0.30475119, 0.65567402, 0.0098742
])

# iniciar com o tempo médio entre cada cliente, começando em 1
taxa_lambda = 1

# iniciar a melhor verossimilhança
melhor_verossimilhanca = 0

# realizar hill climbing e ajustar a taxa lambda
# para melhorar a verossimilhança conjunta
for i in range(?):
    ajuste_aleatorio = np.random.normal(loc=?, scale=?, size=1)[0]
    taxa_lambda += ?

    nova_verossimilhanca = expon.pdf(?, scale=1 / taxa_lambda).prod()
    if ? < ?:
        melhor_verossimilhanca = nova_verossimilhanca
    else:
        taxa_lambda -= ?

print(taxa_lambda)

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

Você deve encontrar aproximadamente 2,21 clientes por hora, considerando esses intervalos. Use a técnica de hill climbing e a estimativa de máxima verossimilhança da mesma forma que fizemos para a distribuição normal para resolver o parâmetro lambda $ \lambda $.

In [ ]:
import numpy as np
from scipy.stats import expon

# tempos observados entre cada cliente
X = np.array([0.27922493, 0.44124056, 0.50967118, 0.44413533, 0.67243048, 0.01870771,
              0.08661839, 0.29967495, 1.68386979, 0.30475119, 0.65567402, 0.0098742
])

# começar com o tempo médio entre cada cliente, iniciando em 1
taxa_lambda = 1

# iniciar a melhor verossimilhança
melhor_verossimilhanca = 0

# realizar hill climbing e ajustar a taxa lambda
# para melhorar a verossimilhança conjunta
for i in range(10_000):
    ajuste_aleatorio = np.random.normal(loc=0, scale=1, size=1)[0]
    taxa_lambda += ajuste_aleatorio

    nova_verossimilhanca = expon.pdf(X, scale=1 / taxa_lambda).prod()
    if melhor_verossimilhanca < nova_verossimilhanca:
        melhor_verossimilhanca = nova_verossimilhanca
    else:
        taxa_lambda -= ajuste_aleatorio

print(taxa_lambda)